In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from influxdb import *
# import Config as cg
from influxdb import DataFrameClient
from datetime import datetime, timedelta
import pytz
from math import sqrt
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


class PowerFactorLoss_KPI():

    def __init__(self, base_pf):
        self.base_pf = base_pf

    def read_Data(self):
        INFLUX_DB_IP = '10.12.97.178'
        INFLUX_DB_PORT = 8086
        INFLUX_DB = 'demoDb'
        TARGET_MEASUREMENT = 'EM_live'
        con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
        query = 'select * from ' + TARGET_MEASUREMENT  + ' where time > now() - 24h '
        df = pd.DataFrame(con_obj.query(query).get_points())
        df['time'] = df['time'].astype('datetime64[ns]')
        df['time'] = df['time'] + timedelta(hours=5, minutes=30)
        
        date = pd.DataFrame([pd.Timestamp(df.time.min())], columns=['date_min'])
        date['new_date'] = date['date_min'].dt.floor('30min')
        start = (date['new_date'] - timedelta(minutes=0))[0]
        print(start)

#         date['end_now'] = [pd.Timestamp(df.time.max())]
        date['end_now'] = [pd.Timestamp(pd.datetime.now())]
        date['end_now'] = date['end_now'].dt.floor('30min')
        end = (date['end_now'] + timedelta(hours=0))[0]
        print(end)

        df = df[(df['time'] >= start) & (df['time'] < end)]
        print(df.time.min())
        print(df.time.max())
        return df


    def __call__(self):
        self.output()

    def calculated_columns_df(self, df):
        
        print(len(df))
        if df['EM_Active Power (kW)'].min() < 0:
            print(len(df))
            print("Energy Generated")
            self.base_pf = -0.99

        df['S'] = df['EM_Active Power (kW)'] / df['EM_Power Factor']
        df['S1_pf'] = df['EM_Active Power (kW)'] / self.base_pf
        df['running_current'] = (df['S'] * 1000) / (sqrt(3) * df['mean_volt'])
        df['current_rms'] = df.apply(lambda x: x['running_current'] / sqrt(1 + (x['Mean_THD'] * x['Mean_THD'])), axis=1)
        df['voltage_rms'] = df.apply(lambda x: x['mean_volt'] / sqrt(1 + (x['EM_THD Voltage'] * x['EM_THD Voltage'])),
                                     axis=1)
        df['PCWH'] = (sqrt(3) * df['current_rms'] * df['voltage_rms']) / 1000
        return df

    def calculated_columns_df_out(self, df):
        print(len(df))
        df_out = pd.DataFrame()
        df_out['S'] = df[['S']].mean()
        df_out['S1_pf'] = df[['S1_pf']].mean()[0]
        df_out['running_current'] = df[['running_current']].mean()[0]
        df_out['current_rms'] = df[['current_rms']].mean()[0]
        df_out['voltage_rms'] = df[['voltage_rms']].mean()[0]
        df_out['PCWH'] = df[['PCWH']].mean()[0]
        return df_out

    def vol_imb_calculated_columns(self, df):
        df['Vmax'] = df[['EM_Voltage Ph1-Ph2 (V)', 'EM_Voltage Ph2-Ph3 (V)', 'EM_Voltage Ph1-Ph3 (V)']].max(axis=1)
        df['Vmin'] = df[['EM_Voltage Ph1-Ph2 (V)', 'EM_Voltage Ph2-Ph3 (V)', 'EM_Voltage Ph1-Ph3 (V)']].min(axis=1)
        df['Vimb'] = np.where((df['Vmax'] - df['mean_volt']) > (df['mean_volt'] - df['Vmin']),
                              (df['Vmax'] - df['mean_volt']), (df['mean_volt'] - df['Vmin']))
        df['Vimb%'] = (df['Vimb'] / df['mean_volt']) * 100
        return df

    def cur_imb_calculated_columns(self, df):
        df['Imax'] = df[['EM_Current Ph1 (A)', 'EM_Current Ph2 (A)', 'EM_Current Ph3 (A)']].max(axis=1)
        df['Imin'] = df[['EM_Current Ph1 (A)', 'EM_Current Ph2 (A)', 'EM_Current Ph3 (A)']].min(axis=1)
        df['Iimb'] = np.where((df['Imax'] - df['mean_current']) > (df['mean_current'] - df['Imin']),
                              (df['Imax'] - df['mean_current']), (df['mean_current'] - df['Imin']))
        df['Iimb%'] = (df['Iimb'] / df['mean_current']) * 100
        return df

    def heat_loss(self, df):
        df['heat_loss'] = 2 * (df['Vimb%'] ** 2) * 100
        return df

    def fun_new(self, df):
        df = df.reset_index()
        print(len(df))
        df = self.calculated_columns_df(df)
        df_out = df.groupby('DeviceID').apply(self.calculated_columns_df_out)

        x = pd.DataFrame(df.groupby('DeviceID')['Duration'].sum())
        x['duration_hrs'] = x['Duration'].dt.total_seconds() / 3600
        x = x.drop('Duration', axis=1).reset_index()
        df_out = df_out.merge(x, on="DeviceID", how="outer")

        df_out['kvah_loss_thd'] = (df_out['S'] - df_out['PCWH']) * df_out['duration_hrs']
        df_out['kvah_loss_pf'] = (df_out['S'] - df_out['S1_pf']) * df_out['duration_hrs']
        df_out['energy_kvah'] = df_out['S'] * df_out['duration_hrs']

        average_pf = df.groupby('DeviceID')['EM_Power Factor'].mean()
        df_out = df_out.merge(average_pf, on='DeviceID', how="outer")
        df_out.rename(columns={'EM_Power Factor': 'average_pf'}, inplace=True)

        average_thd = df.groupby('DeviceID')['Mean_THD'].mean()
        df_out = df_out.merge(average_thd, on='DeviceID', how="outer")
        df_out.rename(columns={'Mean_THD': 'average_thd'}, inplace=True)
        df_out['average_thd'] = df_out['average_thd'] * 100.0

        min_power = df.groupby('DeviceID')['EM_Active Power (kW)'].min()
        df_out = df_out.merge(min_power, on='DeviceID', how="outer")
        df_out.rename(columns={'EM_Active Power (kW)': 'min_power'}, inplace=True)

        min_pf = df.groupby('DeviceID')['EM_Power Factor'].min()
        df_out = df_out.merge(min_pf, on='DeviceID', how="outer")
        df_out.rename(columns={'EM_Power Factor': 'min_pf'}, inplace=True)

        df = self.vol_imb_calculated_columns(df)
        df = self.cur_imb_calculated_columns(df)
        df = self.heat_loss(df)
        imbalance = pd.DataFrame()
        imbalance['Vimb%'] = df.groupby('DeviceID')['Vimb%'].mean()
        imbalance['Iimb%'] = df.groupby('DeviceID')['Iimb%'].mean()
        imbalance['heat_loss'] = df.groupby('DeviceID')['heat_loss'].mean()
        df_out = df_out.merge(imbalance, on='DeviceID', how="outer")
        return df_out

    def fun_main(self, df):
        print(len(df))
        df['EM_Active Power (kW)'] = np.where((df['EM_Power Factor'] == 0), 0, df['EM_Active Power (kW)'])
        df['Mean_THD'] = df['Mean_THD'] / 100
        df['EM_THD Voltage'] = df['EM_THD Voltage'] / 100
        df.set_index('time', inplace=True)
        df_out = df.groupby(pd.Grouper(freq='30T')).apply(self.fun_new)
        df_out = df_out.reset_index()
        df_out['time'] = df_out['time'] + timedelta(minutes=30)
        df_out = df_out[df_out['time'] > (df_out['time'].min())]
        df_out = df_out.drop('level_1', axis=1).set_index('time')
        df_out = df_out.fillna(0)
        return df_out

    def output(self):
            df = self.read_Data()
            df['Duration'] = df.groupby('DeviceID')['time'].diff().fillna(method="bfill")
            df['flag'] = np.where((df['EM_Active Power (kW)'] < 0), 1, 0)
            df_ec = df[df['flag'] == 0]
#             df_c = self.fun_main(df_ec)
#             df_c.index = df_c.index + timedelta(hours=5, minutes=30)
#             print(df_c.tail())
            # print(self.DFDBClient.write_points(df_c, cg.WRITE_MEASUREMENT, tag_columns=['DeviceID']))
            print('Inside half hour scheduling')

            df_eg = df[df['flag'] == 1]
            if not df_eg.empty:
                print("Energy Generated : Output 2")
                print(df_eg[df_eg['time'] < '2020-09-12 07:00:00'])
                df_g = self.fun_main(df_eg)
                print(df_g.tail())
                # print(self.DFDBClient.write_points(df_g, cg.WRITE_MEASUREMENT_2, tag_columns=['DeviceID']))


if __name__ == '__main__':
    cat = PowerFactorLoss_KPI(0.99)
    cat.output()

2020-09-11 12:00:00
2020-09-12 12:00:00
2020-09-11 12:03:08.730274048
2020-09-12 11:59:59.952448
Inside half hour scheduling
Energy Generated : Output 2
                                time DeviceID  EM_Active Power (kW)  \
145558 2020-09-11 21:49:36.885472000      EM7                 -0.01   
286323 2020-09-12 06:56:35.685990144      EM7                 -0.09   
286335 2020-09-12 06:56:38.408143872      EM7                 -0.22   
286345 2020-09-12 06:56:40.785334016      EM7                 -0.17   
286358 2020-09-12 06:56:43.570080000      EM7                 -0.12   
286369 2020-09-12 06:56:45.963006976      EM7                 -0.07   

        EM_Current Avg (A)  EM_Current Ph1 (A)  EM_Current Ph2 (A)  \
145558                0.15                0.15                0.15   
286323                0.33                0.33                0.33   
286335                0.35                0.34                0.34   
286345                0.28                0.27                0.28   

ValueError: Wrong number of items passed 34, placement implies 1